Aluna: Cristina Freitas

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd

filename_features = "UCI HAR Dataset/features.txt"
filename_labels = "UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "UCI HAR Dataset/train/y_train.txt"

filename_subtest = "UCI HAR Dataset/test/subject_test.txt"
filename_xtest = "UCI HAR Dataset/test/X_test.txt"
filename_ytest = "UCI HAR Dataset/test/y_test.txt"

# Removido squeeze=True
features_df = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
features = features_df['nome_var']  # Convertido manualmente para Series

labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train_df = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_train = subject_train_df['subject_id']  # Convertido para Series

X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test_df = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
subject_test = subject_test_df['subject_id']  # Convertido para Series

X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

In [6]:
%%time

 clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001).fit(X_train, y_train)


CPU times: total: 4.53 s
Wall time: 4.75 s


In [12]:
# Previsões
y_train_pred = clf.predict(X_train)
y_valid_pred = clf.predict(X_valid)

# Acurácia
print("Acurácia no Treino:", accuracy_score(y_train, y_train_pred))
print("Acurácia na Validação:", accuracy_score(y_valid, y_valid_pred))

# Matriz de confusão
print("\nMatriz de Confusão (Validação):")
print(confusion_matrix(y_valid, y_valid_pred))

# Relatório de classificação
print("\nRelatório de Classificação (Validação):")
print(classification_report(y_valid, y_valid_pred))

Acurácia no Treino: 0.9775117881755532
Acurácia na Validação: 0.9287268770402611

Matriz de Confusão (Validação):
[[307  13   9   0   0   0]
 [ 17 230  17   0   0   0]
 [  6   5 243   0   0   0]
 [  0   0   0 294  35   0]
 [  0   0   0  29 316   0]
 [  0   0   0   0   0 317]]

Relatório de Classificação (Validação):
              precision    recall  f1-score   support

           1       0.93      0.93      0.93       329
           2       0.93      0.87      0.90       264
           3       0.90      0.96      0.93       254
           4       0.91      0.89      0.90       329
           5       0.90      0.92      0.91       345
           6       1.00      1.00      1.00       317

    accuracy                           0.93      1838
   macro avg       0.93      0.93      0.93      1838
weighted avg       0.93      0.93      0.93      1838



## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [13]:
%%time
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_valida = prcomp.transform(X_valid)
pc_teste  = prcomp.transform(X_test)

pc_treino.shape

CPU times: total: 328 ms
Wall time: 204 ms


(5514, 1)

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [14]:
%%time
# Quantidades de componentes a serem testadas
componentes_list = [1, 2, 5, 10, 50]

# Dicionário para armazenar os resultados
resultados = {}

# Loop para testar com diferentes quantidades de componentes
for n_components in componentes_list:
    print(f"\nTreinando com {n_components} componentes principais:")
    
    # 1. Realiza o PCA com n_components
    pca = PCA(n_components=n_components)
    
    pc_treino = pca.fit_transform(X_train)
    pc_valida = pca.transform(X_valid)
    pc_teste  = pca.transform(X_test)
    
    # 2. Treina a árvore de decisão
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)
    
    # 3. Faz as previsões
    y_train_pred = clf.predict(pc_treino)
    y_valid_pred = clf.predict(pc_valida)
    
    # 4. Avalia o desempenho
    acuracia_treino = accuracy_score(y_train, y_train_pred)
    acuracia_validacao = accuracy_score(y_valid, y_valid_pred)
    
    # 5. Matriz de confusão e relatório de classificação
    matriz_confusao = confusion_matrix(y_valid, y_valid_pred)
    relatorio_classificacao = classification_report(y_valid, y_valid_pred)
    
    # 6. Armazena os resultados
    resultados[n_components] = {
        "acuracia_treino": acuracia_treino,
        "acuracia_validacao": acuracia_validacao,
        "matriz_confusao": matriz_confusao,
        "relatorio_classificacao": relatorio_classificacao
    }

    # Exibe as métricas para o conjunto de validação
    print(f"Acurácia no Treino (com {n_components} componentes): {acuracia_treino:.4f}")
    print(f"Acurácia na Validação (com {n_components} componentes): {acuracia_validacao:.4f}")
    print(f"\nMatriz de Confusão (com {n_components} componentes):\n{matriz_confusao}")
    print(f"\nRelatório de Classificação (com {n_components} componentes):\n{relatorio_classificacao}")


Treinando com 1 componentes principais:
Acurácia no Treino (com 1 componentes): 0.4949
Acurácia na Validação (com 1 componentes): 0.4940

Matriz de Confusão (com 1 componentes):
[[149  77 103   0   0   0]
 [ 97  98  69   0   0   0]
 [ 40  10 204   0   0   0]
 [  0   0   0  52 262  15]
 [  0   0   0  27 318   0]
 [  0   0   0  43 187  87]]

Relatório de Classificação (com 1 componentes):
              precision    recall  f1-score   support

           1       0.52      0.45      0.48       329
           2       0.53      0.37      0.44       264
           3       0.54      0.80      0.65       254
           4       0.43      0.16      0.23       329
           5       0.41      0.92      0.57       345
           6       0.85      0.27      0.42       317

    accuracy                           0.49      1838
   macro avg       0.55      0.50      0.46      1838
weighted avg       0.55      0.49      0.46      1838


Treinando com 2 componentes principais:
Acurácia no Treino (com 2

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

O que aconteceu com a acurária? 

Quando temos poucas variáveis (exemplo: n=1) podemos perder informação e não temos como observar a complexidade do modelo. 
Como efeito disso podemos ter overfitting ou underfitting pois com poucos dados o modelo tem dificuldade de se adaptar. 
Contudo, com muita informação (exemplo: n=50) se consegue capturar as complexidades do modelo, porém o excesso tende ao 
overfitting (ou excesso de adaptação)



O que aconteceu com o tempo de processamento?

Quanto maior a quantidade de dados maior o tempo de processamento e quanto menor a quantidade de dados menor o tempo de 
processamento. No entanto, se o número de componentes for pequeno pode-se perder uma representação significativa dos dados.



O que seria o melhor dos mundos?

Poderíamos usar a validação cruzada para observar o comportamento da acurária no contexto das quantidades de variáveis.
Outra coisa seria ajustar o parâmetro ccp_alpha para regular a complexidade e evitar o overfitting e utilizar um gráfico 
onde mostre acurária x número de componentes para saber o tempo ótimo de equilíbrio entre desempenho e tempo.






